In [71]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib

In [73]:
filepath = "../data/cr_loan_preprocessed.csv"
df = pd.read_csv(filepath)
df

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
1,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
2,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
3,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4
4,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,0.25,N,2
...,...,...,...,...,...,...,...,...,...,...,...,...
32404,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,0,0.11,N,30
32405,54,120000,MORTGAGE,4.0,PERSONAL,A,17625,7.49,0,0.15,N,19
32406,65,76000,RENT,3.0,HOMEIMPROVEMENT,B,35000,10.99,1,0.46,N,28
32407,56,150000,MORTGAGE,5.0,PERSONAL,B,15000,11.48,0,0.10,N,26


In [74]:
print(df.dtypes)
print(40 * '-')
for categorical in df.select_dtypes("object"):
    print(f"{categorical} - {df[categorical].unique()}")

person_age                      int64
person_income                   int64
person_home_ownership          object
person_emp_length             float64
loan_intent                    object
loan_grade                     object
loan_amnt                       int64
loan_int_rate                 float64
loan_status                     int64
loan_percent_income           float64
cb_person_default_on_file      object
cb_person_cred_hist_length      int64
dtype: object
----------------------------------------
person_home_ownership - ['OWN' 'MORTGAGE' 'RENT' 'OTHER']
loan_intent - ['EDUCATION' 'MEDICAL' 'VENTURE' 'PERSONAL' 'HOMEIMPROVEMENT'
 'DEBTCONSOLIDATION']
loan_grade - ['B' 'C' 'A' 'D' 'E' 'F' 'G']
cb_person_default_on_file - ['N' 'Y']


In [75]:
default_on_file_encoding = {"N": 0, "Y": 1}
loan_grade_encoding = {'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7}

df['cb_person_default_on_file'] = df['cb_person_default_on_file'].replace(default_on_file_encoding).astype("int64")
df['loan_grade'] = df['loan_grade'].replace(loan_grade_encoding).astype("int64")

df = pd.get_dummies(df, columns=['person_home_ownership', 'loan_intent'], drop_first=True, dtype="int64")

C:\Users\eren.onaran\AppData\Local\Temp\ipykernel_20540\2195929230.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['cb_person_default_on_file'] = df['cb_person_default_on_file'].replace(default_on_file_encoding).astype("int64")
C:\Users\eren.onaran\AppData\Local\Temp\ipykernel_20540\2195929230.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['loan_grade'] = df['loan_grade'].replace(loan_grade_encoding).astype("int64")


In [76]:
print(df.dtypes)
print(40 * '-')
for categorical in df.select_dtypes("object"):
    print(f"{categorical} - {df[categorical].unique()}")

person_age                       int64
person_income                    int64
person_emp_length              float64
loan_grade                       int64
loan_amnt                        int64
loan_int_rate                  float64
loan_status                      int64
loan_percent_income            float64
cb_person_default_on_file        int64
cb_person_cred_hist_length       int64
person_home_ownership_OTHER      int64
person_home_ownership_OWN        int64
person_home_ownership_RENT       int64
loan_intent_EDUCATION            int64
loan_intent_HOMEIMPROVEMENT      int64
loan_intent_MEDICAL              int64
loan_intent_PERSONAL             int64
loan_intent_VENTURE              int64
dtype: object
----------------------------------------


In [77]:
df["emp_length_age_ratio"] = df["person_emp_length"] / df["person_age"]
df["emp_length_cred_hist_ratio"] = df["person_emp_length"] / df["cb_person_cred_hist_length"]
df["income_emp_length_product"] = df["person_emp_length"] * df["person_income"]
df["cred_hist_age_ratio"] = df["cb_person_cred_hist_length"] / df["person_age"]
df["emp_start"] = df["person_age"] - df["person_emp_length"]
df["interest_income_ratio"] = df["loan_amnt"] * df["loan_int_rate"] / df["person_income"]
df["parent_dependency"] = ((df["person_age"] - df["cb_person_cred_hist_length"]) < 18).astype("int64")

In [78]:
print(df.dtypes)
print(40 * '-')
for categorical in df.select_dtypes("object"):
    print(f"{categorical} - {df[categorical].unique()}")

person_age                       int64
person_income                    int64
person_emp_length              float64
loan_grade                       int64
loan_amnt                        int64
loan_int_rate                  float64
loan_status                      int64
loan_percent_income            float64
cb_person_default_on_file        int64
cb_person_cred_hist_length       int64
person_home_ownership_OTHER      int64
person_home_ownership_OWN        int64
person_home_ownership_RENT       int64
loan_intent_EDUCATION            int64
loan_intent_HOMEIMPROVEMENT      int64
loan_intent_MEDICAL              int64
loan_intent_PERSONAL             int64
loan_intent_VENTURE              int64
emp_length_age_ratio           float64
emp_length_cred_hist_ratio     float64
income_emp_length_product      float64
cred_hist_age_ratio            float64
emp_start                      float64
interest_income_ratio          float64
parent_dependency                int64
dtype: object
-----------

In [79]:
df.to_csv("../data/cr_loan_featured.csv", index=False)